<a href="https://colab.research.google.com/github/yk-mt12/project-exercise/blob/master/tabelog_NegaPojiAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install mecab-python3
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null
# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc
!pip install neologdn
!pip install jaconv

import collections
from collections import namedtuple
import jaconv
import MeCab
import neologdn
import numpy as np
import pandas as pd
import re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 35.2 MB/s 
Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 5.8 MB/s 
  Created wheel for neologdn: filename=neologdn-0.5.1-cp37-cp37m-linux_x86_64.whl size=172974 sha256=4b25b6e938e9c8df355f766bb8d7e9fc7617f75225b2eeb21b743f91b5e456c8
  Stored in directory: /root/.cache/pip/wheels/19/15/5c/55b33d02e16129ef81313e4c86e473d6dd1cecf7317a525a9b
Successfully built neologdn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created whe

In [5]:
%cd drive/My Drive

[Errno 2] No such file or directory: 'drive/My Drive'
/content


In [7]:
df = pd.read_csv('./data/review.csv').reset_index()
print(df)
# dic = pd.read_csv('./dic/dic_shop_in_view03.csv').reset_index()
# print(dic)

     index  Unnamed: 0  store_id store_name                 genre  rate  \
0        0           0         1        本湖月  大阪難波駅 322m / 懐石・会席料理  4.57   
1        1           1         1        本湖月  大阪難波駅 322m / 懐石・会席料理  4.57   
2        2           2         1        本湖月  大阪難波駅 322m / 懐石・会席料理  4.57   
3        3           3         1        本湖月  大阪難波駅 322m / 懐石・会席料理  4.57   
4        4           4         1        本湖月  大阪難波駅 322m / 懐石・会席料理  4.57   
..     ...         ...       ...        ...                   ...   ...   
460    460         460         4         米増    福島駅 654m / 懐石・会席料理  4.33   
461    461         461         4         米増    福島駅 654m / 懐石・会席料理  4.33   
462    462         462         4         米増    福島駅 654m / 懐石・会席料理  4.33   
463    463         463         4         米増    福島駅 654m / 懐石・会席料理  4.33   
464    464         464         4         米増    福島駅 654m / 懐石・会席料理  4.33   

     review_cnt                                             review  
0           198  長年の伺いたかった本湖月さ

In [8]:
change_reviews = []
for review in df['review']:
  review = neologdn.normalize(review)
  review = review.replace('@', 'アット')
  review = review.replace('＠', 'アット')
  review = review.replace('■', '')
  review = review.replace('リピ', 'リピート')
  review = review.replace('good', '良い')
  review = review.replace('コスパ', 'コストパフォーマンス')
  review = review.replace('いいです', '良い')
  review = review.replace('よかった', '良い')
  review = review.replace('します', 'する')
  review = review.replace('しません', 'しない')
  review = re.split('[。.！!?？!?♪★☆(´･`)*^]', review)
  change_reviews.append(' '.join(review))
reviews_df = pd.DataFrame(change_reviews, columns=['review'])

In [9]:
#http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt
#上のURL開いてダウンロードする
#テキストファイル読み込み

with open('./data/japanese.txt', 'r', encoding='utf-8') as f:
  stopwords = [w.strip() for w in f]
  stopwords = list(stopwords)
stopwords.append('*')

with open ('./dic/辞書_店_内観0 - 内観.csv', 'r', encoding = 'utf-8') as f:
  #単語・読み仮名・品詞・スコアに分割してリストとして格納
  negapoji_list = [negapoji_word.split(',') for negapoji_word in  f.readlines()]
negapoji_df = pd.DataFrame(negapoji_list)
#jaconvを使って読み仮名を全てカタカナに変換
# negapoji_df['reading'] = negapoji_df['reading'].apply(lambda x : jaconv.hira2kata(x))
#なぜか読みや品詞まで同じなのに、異なるスコアが割り当てられていたものがあったので重複を削除
# negapoji_df = negapoji_df[~negapoji_df[['base_form', 'reading', 'pos']].duplicated()]

In [60]:
class NegaPojiAnalysis():
  def __init__(self, reviews_df, negapoji_list, stopwords):
    self.reviews_df = reviews_df
    self.negapoji_list = negapoji_list
    self.stopwords = stopwords
    self.path = "-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
    self.tagger = MeCab.Tagger(self.path)

  def tokenize(self):
    word_list = []
    for i, review in enumerate(self.reviews_df['review']):
        node = self.tagger.parseToNode(review)
        while node:
          surface = node.surface
          pos = node.feature.split(",")[0]
          pos_detail1 = node.feature.split(",")[1]
          pos_detail2 = node.feature.split(",")[2]
          pos_detail3 = node.feature.split(",")[3]
          infl_type = node.feature.split(",")[4]  
          infl_form = node.feature.split(",")[5]
          base_form = node.feature.split(",")[6]
          if len(node.feature.split(",")) >= 8:
            reading = node.feature.split(",")[7]
          if pos == '名詞' or pos == '動詞' or pos == '形容詞' or pos == '副詞':
            word_list.append([i, surface, pos, pos_detail1, pos_detail2, pos_detail3, infl_type, infl_form, base_form, reading])
          node = node.next
    word_df = pd.DataFrame(word_list, columns = ['index', 'surface', 'pos', 'pos_detail1', 'pos_detail2', 'pos_detail3', 'infl_type', 'infl_form', 'base_form', 'reading'])
    word_df['pos'] = word_df['pos'].apply(lambda negapoji_list : negapoji_list.split(',')[0])
    # print(word_df['base_form'])
    return word_df

  # def generate_negapoji_score(self, word_df, negapoji_df):
  #   print('word_df', word_df)
  #   print('negapoji_df', negapoji_df)

  #   score_result_df = pd.merge(word_df, negapoji_df, on = ['base_form', 'pos', 'reading'], how = 'left')
  #   print(score_result_df)
  #   return score_result_df

  def generate_review_score(self, parse_text_df, negapoji_list):
    result = []
    resultScore = 0
    for i in range(len(parse_text_df)-1):
            # テキストをそれぞれ格納する。
            text = parse_text_df['base_form'][i] # 任意の単語
            if i != 0: # 前単語
                textBefore = parse_text_df['base_form'][i-1]
            else:
                textBefore = ''
            if i == len(parse_text_df):
              textAfter = ''
            else:
              textAfter = parse_text_df['base_form'][i+1] # 後単語 or それ以外
              
            for j in range(len(negapoji_list)-1):
                #  print(col[col[1].isin([text]) and col[2].isin([textBefore]) and col[3].isin([textAfter])])
                # if negapoji_list[j][1] == text and (negapoji_list[j][2] == textBefore and negapoji_list[j][3] == textAfter):
                #   resultScore += negapoji_list[j][4]
                # # elif negapoji_list[j][1] == text and (negapoji_list[j][2] == textBefore and negapoji_list[j][3] != textAfter):
                # #   resultScore += negapoji_list[j][4]
              
            # break
    print(resultScore) 
        # temp_df = self.negapoji_list[i]
        # text = self.reviews_df['review'][i]
        #text = ''.join(list(temp_df['word']))
        # score = temp_df['score'].astype(float).sum()
        #スコアをスコアが付与されている単語数で割った値を算出
        # score_r = score/temp_df['score'].astype(float).count()
        # result.append([i, text, score,score_r])
    result_df = pd.DataFrame(result, columns = ['index', 'review', '累計スコア', '標準化スコア']).sort_values(by = '標準化スコア', ascending=False).reset_index(drop = True)
    return result_df

In [61]:
negapoji_analysis = NegaPojiAnalysis(reviews_df, negapoji_list, stopwords)
parse_text_df = negapoji_analysis.tokenize()
# score_result_df = negapoji_analysis.generate_negapoji_score(parse_text_df, negapoji_df)

review_result_df = negapoji_analysis.generate_review_score(parse_text_df, negapoji_list)

0


In [ ]:
review_result_df